In [1]:
# Import libraries
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

In [2]:
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

In [24]:
!pip3 install bayesian-optimization==1.4.0

Defaulting to user installation because normal site-packages is not writeable
  Using cached bayesian_optimization-1.4.0-py3-none-any.whl (17 kB)
  Attempting uninstall: bayesian-optimization
    Found existing installation: bayesian-optimization 1.4.3
    Uninstalling bayesian-optimization-1.4.3:
      Successfully uninstalled bayesian-optimization-1.4.3


In [25]:
!pip3 install colorama==0.4.4

Defaulting to user installation because normal site-packages is not writeable


In [26]:
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from bayes_opt import BayesianOptimization

In [27]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [28]:
# from google.colab import drive
# drive.mount('/content/drive')

In [29]:
 # read the csv files
df_features = pd.read_csv("1677CVEs_783dim_features_20230326.csv")
df_features.rename({"Unnamed: 0" : "CVE"}, inplace = True, axis = 1)
df_features.set_index('CVE', inplace = True)
df_features

,count__variance_larger_than_standard_deviation,count__has_duplicate_max,count__has_duplicate_min,count__has_duplicate,count__sum_values,count__abs_energy,count__mean_abs_change,count__mean_change,count__mean_second_derivative_central,count__median,...,count__fourier_entropy__bins_5,count__fourier_entropy__bins_10,count__fourier_entropy__bins_100,count__permutation_entropy__dimension_3__tau_1,count__permutation_entropy__dimension_4__tau_1,count__permutation_entropy__dimension_5__tau_1,count__permutation_entropy__dimension_6__tau_1,count__permutation_entropy__dimension_7__tau_1,count__query_similarity_count__query_None__threshold_0.0,count__mean_n_absolute_max__number_of_maxima_7
CVE,,,,,,,,,,,,,,,,,,,,,
CVE-2006-10001,1.0,0.0,1.0,1.0,16.0,60.0,2.000000,-0.400000,0.125000,2.5,...,1.039721,1.039721,1.386294,1.386294,1.098612,0.693147,-0.000000,NaN,NaN,NaN
CVE-2008-10002,0.0,1.0,1.0,1.0,15.0,37.0,1.333333,-0.333333,0.100000,2.0,...,1.039721,1.039721,1.039721,1.054920,1.039721,1.098612,0.693147,-0.000000,NaN,NaN
CVE-2008-10003,1.0,0.0,1.0,1.0,16.0,44.0,1.000000,-0.333333,-0.100000,2.0,...,1.386294,1.386294,1.386294,1.332179,1.386294,1.098612,0.693147,-0.000000,NaN,NaN
CVE-2008-10004,1.0,0.0,1.0,1.0,20.0,110.0,4.000000,-0.333333,0.200000,1.0,...,1.039721,1.386294,1.386294,1.054920,1.039721,1.098612,0.693147,-0.000000,NaN,NaN
CVE-2011-10002,1.0,0.0,1.0,1.0,12.0,54.0,2.333333,-1.000000,-0.500000,2.5,...,1.098612,1.098612,1.098612,0.693147,-0.000000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CVE-2023-28685,0.0,0.0,1.0,1.0,15.0,35.0,1.285714,-0.142857,0.000000,2.0,...,1.332179,1.332179,1.609438,1.329661,1.609438,1.386294,1.098612,0.693147,NaN,2.000000
CVE-2023-28686,1.0,1.0,1.0,1.0,15.0,43.0,1.666667,0.000000,-0.100000,2.0,...,1.039721,1.039721,1.386294,1.609438,1.386294,1.098612,0.693147,-0.000000,NaN,NaN
CVE-2023-28708,1.0,0.0,1.0,1.0,30.0,100.0,1.692308,-0.153846,-0.125000,1.0,...,1.494175,1.732868,1.906155,1.329661,1.893788,2.302585,2.197225,2.079442,NaN,3.285714


In [30]:
df_flag = pd.read_csv("CVE_list_with_flagCR_20230326.csv", index_col = ['CVE'] )
df_flag_sorted = df_flag.sort_values('CVE')
df_flag_sorted.drop('Unnamed: 0', inplace = True, axis = 1)
df_flag_sorted

,flag_CR
CVE,
CVE-2006-10001,0
CVE-2008-10002,0
CVE-2008-10003,1
CVE-2008-10004,1
CVE-2011-10002,1
...,...
CVE-2023-28685,0
CVE-2023-28686,0
CVE-2023-28708,0


## Filter the features which are NaN

In [31]:
df_features = impute(df_features)
df_features

/home/kimura/.local/lib/python3.10/site-packages/tsfresh/utilities/dataframe_functions.py:198: RuntimeWarning: The columns ['count__fft_coefficient__attr_"real"__coeff_25'
 'count__fft_coefficient__attr_"real"__coeff_26'
 'count__fft_coefficient__attr_"real"__coeff_27'
 'count__fft_coefficient__attr_"real"__coeff_28'
 'count__fft_coefficient__attr_"real"__coeff_29'
 'count__fft_coefficient__attr_"real"__coeff_30'
 'count__fft_coefficient__attr_"real"__coeff_31'
 'count__fft_coefficient__attr_"real"__coeff_32'
 'count__fft_coefficient__attr_"real"__coeff_33'
 'count__fft_coefficient__attr_"real"__coeff_34'
 'count__fft_coefficient__attr_"real"__coeff_35'
 'count__fft_coefficient__attr_"real"__coeff_36'
 'count__fft_coefficient__attr_"real"__coeff_37'
 'count__fft_coefficient__attr_"real"__coeff_38'
 'count__fft_coefficient__attr_"real"__coeff_39'
 'count__fft_coefficient__attr_"real"__coeff_40'
 'count__fft_coefficient__attr_"real"__coeff_41'
 'count__fft_coefficient__attr_"real"__coeff

,count__variance_larger_than_standard_deviation,count__has_duplicate_max,count__has_duplicate_min,count__has_duplicate,count__sum_values,count__abs_energy,count__mean_abs_change,count__mean_change,count__mean_second_derivative_central,count__median,...,count__fourier_entropy__bins_5,count__fourier_entropy__bins_10,count__fourier_entropy__bins_100,count__permutation_entropy__dimension_3__tau_1,count__permutation_entropy__dimension_4__tau_1,count__permutation_entropy__dimension_5__tau_1,count__permutation_entropy__dimension_6__tau_1,count__permutation_entropy__dimension_7__tau_1,count__query_similarity_count__query_None__threshold_0.0,count__mean_n_absolute_max__number_of_maxima_7
CVE,,,,,,,,,,,,,,,,,,,,,
CVE-2006-10001,1.0,0.0,1.0,1.0,16.0,60.0,2.000000,-0.400000,0.125000,2.5,...,1.039721,1.039721,1.386294,1.386294,1.098612,0.693147,-0.000000,0.693147,0.0,2.285714
CVE-2008-10002,0.0,1.0,1.0,1.0,15.0,37.0,1.333333,-0.333333,0.100000,2.0,...,1.039721,1.039721,1.039721,1.054920,1.039721,1.098612,0.693147,-0.000000,0.0,2.285714
CVE-2008-10003,1.0,0.0,1.0,1.0,16.0,44.0,1.000000,-0.333333,-0.100000,2.0,...,1.386294,1.386294,1.386294,1.332179,1.386294,1.098612,0.693147,-0.000000,0.0,2.285714
CVE-2008-10004,1.0,0.0,1.0,1.0,20.0,110.0,4.000000,-0.333333,0.200000,1.0,...,1.039721,1.386294,1.386294,1.054920,1.039721,1.098612,0.693147,-0.000000,0.0,2.285714
CVE-2011-10002,1.0,0.0,1.0,1.0,12.0,54.0,2.333333,-1.000000,-0.500000,2.5,...,1.098612,1.098612,1.098612,0.693147,-0.000000,0.693147,0.693147,0.693147,0.0,2.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CVE-2023-28685,0.0,0.0,1.0,1.0,15.0,35.0,1.285714,-0.142857,0.000000,2.0,...,1.332179,1.332179,1.609438,1.329661,1.609438,1.386294,1.098612,0.693147,0.0,2.000000
CVE-2023-28686,1.0,1.0,1.0,1.0,15.0,43.0,1.666667,0.000000,-0.100000,2.0,...,1.039721,1.039721,1.386294,1.609438,1.386294,1.098612,0.693147,-0.000000,0.0,2.285714
CVE-2023-28708,1.0,0.0,1.0,1.0,30.0,100.0,1.692308,-0.153846,-0.125000,1.0,...,1.494175,1.732868,1.906155,1.329661,1.893788,2.302585,2.197225,2.079442,0.0,3.285714


## Train-test split

In [32]:
train_data = df_features
y = df_flag_sorted['flag_CR']
X = train_data.values

# stratify　= y: 訓練データとテストデータの陽性陰性の比率が同一になるように分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle = True, stratify = y, random_state = 1234)

## Model

In [33]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

## Scores

In [34]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.93      0.78      0.85       445
           1       0.25      0.54      0.34        59

    accuracy                           0.75       504
   macro avg       0.59      0.66      0.59       504
weighted avg       0.85      0.75      0.79       504



In [35]:
print("Accuracy:{:.3f}".format(accuracy_score(y_test, y_pred)))
print("Precision:{:.3f}".format(precision_score(y_test, y_pred)))
print("Recall:{:.3f}".format(recall_score(y_test, y_pred)))
print("F1 Score:{:.3f}".format(f1_score(y_test, y_pred)))

Accuracy:0.754
Precision:0.542
Recall:0.248
F1 Score:0.340


# Hyper Parameter tuning

In [36]:
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [37]:
# Define the stratified_kfold_score function for bayesian OP
def stratified_kfold_score(clf,X,y,n_fold):
    #X,y = X,y
    strat_kfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=1234)
    f1score_list = []

    for train_index, test_index in strat_kfold.split(X, y):
        x_train_fold, x_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]
        clf.fit(x_train_fold, y_train_fold)
        y_pred = clf.predict(x_test_fold)
        f1score_test = f1_score(y_pred,y_test_fold)
        f1score_list.append(f1score_test)

    return np.array(f1score_list).mean()

In [38]:
# Define the function to maximize
def bo_params_rf(max_samples,n_estimators,max_features):
    
    params = {
        'max_samples': max_samples,
        'max_features':max_features,
        'n_estimators':int(n_estimators)
    }
    clf = RandomForestClassifier(max_samples=params['max_samples'],max_features=params['max_features'],n_estimators=params['n_estimators'])
    score = stratified_kfold_score(clf,X_train, y_train, 10)
    return score

In [39]:
rf_bo = BayesianOptimization(f=bo_params_rf, pbounds={
                                              "max_samples": (0.0, 1.0),
                                                'max_features':(0.5,1),
                                              'n_estimators':(200,2000)
                                             },random_state=1234, verbose=2)

In [ ]:
results = rf_bo.maximize(n_iter=200, init_points=20)

|   iter    |  target   | max_fe... | max_sa... | n_esti... |
-------------------------------------------------------------
| 1         | 0.2397    | 0.5958    | 0.6221    | 987.9     |
| 2         | 0.2355    | 0.8927    | 0.78      | 690.7     |
| 3         | 0.241     | 0.6382    | 0.8019    | 1.925e+03 |
| 4         | 0.1777    | 0.938     | 0.3578    | 1.102e+03 |
| 5         | 0.2466    | 0.8417    | 0.7127    | 866.5     |
| 6         | 0.2292    | 0.7806    | 0.5031    | 224.8     |
| 7         | 0.2568    | 0.8864    | 0.8826    | 856.8     |
| 8         | 0.07305   | 0.8077    | 0.07538   | 863.9     |
| 9         | 0.2384    | 0.9666    | 0.6514    | 915.0     |
| 10        | 0.1325    | 0.8944    | 0.3168    | 1.223e+03 |
| 11        | 0.209     | 0.9346    | 0.4362    | 1.644e+03 |
| 12        | 0.2397    | 0.5719    | 0.7043    | 1.468e+03 |
| 13        | 0.2531    | 0.6094    | 0.9249    | 995.9     |
| 14        | 0.06087   | 0.9547    | 0.05981   | 531.7     |
| 15    

| 80        | 0.2544    | 0.5581    | 0.6614    | 986.7     |
| 81        | 0.03774   | 0.9207    | 0.0469    | 987.4     |
| 82        | 0.2563    | 0.5583    | 0.8942    | 986.1     |
| 83        | 0.1288    | 0.5418    | 0.2354    | 985.9     |
| 84        | 0.2479    | 0.7514    | 0.5305    | 211.1     |
| 85        | 0.2501    | 0.7359    | 0.9999    | 986.6     |
| 86        | 0.2649    | 0.9798    | 0.9303    | 212.1     |
| 87        | 0.256     | 0.5       | 1.0       | 1.625e+03 |
| 88        | 0.2435    | 0.5507    | 0.6141    | 725.2     |
| 89        | 0.2447    | 0.9405    | 0.9783    | 871.8     |
| 90        | 0.1299    | 0.8355    | 0.2528    | 871.6     |
| 91        | 0.03127   | 0.5655    | 0.03928   | 725.7     |
| 92        | 0.2515    | 0.5482    | 0.8041    | 992.7     |
| 93        | 0.2485    | 0.9989    | 0.8854    | 725.4     |
| 94        | 0.2548    | 0.9232    | 0.9403    | 724.7     |
| 95        | 0.1293    | 0.739     | 0.2672    | 992.4     |
| 96    

| 161       | 0.2107    | 0.8267    | 0.5357    | 903.4     |
| 162       | 0.2569    | 0.5728    | 0.9033    | 974.8     |
| 163       | 0.2362    | 0.8076    | 0.6548    | 1.48e+03  |
| 164       | 0.2549    | 0.7996    | 0.7225    | 974.3     |
| 165       | 0.1095    | 0.5055    | 0.2559    | 661.7     |
| 166       | 0.2562    | 0.7126    | 0.8584    | 1.076e+03 |
| 167       | 0.06733   | 0.6507    | 0.09221   | 1.076e+03 |
| 168       | 0.08935   | 0.8855    | 0.121     | 415.1     |
| 169       | 0.08939   | 0.6252    | 0.1392    | 973.9     |
| 170       | 0.09414   | 0.5253    | 0.2286    | 698.9     |
| 171       | 0.2562    | 0.9886    | 0.9555    | 1.076e+03 |
| 172       | 0.266     | 0.6369    | 0.9705    | 1.716e+03 |
| 173       | 0.1695    | 0.6212    | 0.3618    | 1.716e+03 |
| 174       | 0.1276    | 0.8722    | 0.2603    | 255.7     |
| 175       | 0.2601    | 0.8267    | 0.9397    | 1.626e+03 |
| 176       | 0.2397    | 0.9748    | 0.8854    | 1.716e+03 |
| 177   

NameError: name 'rf_bo' is not defined